Binary Logistic Regression 0 Graduate School Example
================================================
#### Description of the data
For this analysis below, we are going to consider the factors for being accepted into graduate
school.
    
A researcher is interested in how variables, such as GRE (Graduate Record Exam
scores), GPA (grade point average) and prestige of the undergraduate institution,
effect admission into graduate school. The response variable, admit/don't admit,
is a binary variable.


In [ ]:
mydata <- read.csv("binary.csv")
## view the first few rows of the data
head(mydata)
## admit gre gpa rank
## 1 0 380 3.61 3
## 2 1 660 3.67 3
## 3 1 800 4.00 1
## 4 1 640 3.19 4
## 5 0 520 2.93 4
## 6 1 760 3.00 2

* This dataset has a binary response (outcome, dependent) variable called admit. There
are three predictor variables: gre, gpa and rank.
* We will treat the variables gre and gpa as continuous. The variable rank takes on the
values 1 through 4.
* Institutions with a rank of 1 have the highest prestige, while those with a rank of 4
have the lowest.
* We can get basic descriptives for the entire data set by using summary.


In [ ]:
summary(mydata)




Now we will construct a two-way contingency table of categorical outcome and pre-
dictors we want to make sure there are not 0 cells

In [ ]:
table(mydata$admit)
#
# 0 1
#273 127

In [ ]:

xtabs(~admit + rank, data = mydata)
## rank
## admit 1 2 3 4
## 0 28 97 93 55
## 1 33 54 28 12
27

### Using the logit model
* The code below estimates a logistic regression model using the **``glm()``** (generalized linear
model) function.
* Importantly, we convert rank to a factor to indicate that rank should be treated as a
factor (categorical variable). We dont need to respecify ordering in this case.




In [ ]:
mydata$rank <- factor(mydata$rank)
model1 <- glm(admit ~ gre + gpa + rank,
                data = mydata,
                family = "binomial")

In [ ]:
### Using the **``summary()``** function
* Since we gave our model a name ("model1"), R will not produce any output from our
regression.
* In order to get the results we use the **``summary()``** command:

In [ ]:
summary(model1)
# tidy(model1)
# glance(model1)

### Interpreting the summary output
* The rst output we see is the deviance residuals, which are a measure of model t. This
part of output shows the distribution of the deviance residuals for individual cases used
in the model.
Later we discuss how to use summaries of the deviance statistic to assess model t.
* The next part of the output shows the coecients, their standard errors, the z-statistic
(sometimes called a Wald z-statistic), and the associated p-values.
* Both gre and gpa are statistically signicant, as are the three terms for rank. The
logistic regression coecients give the change in the log odds of the outcome for a one
unit increase in the predictor variable.
* gre: Similarly, for every one unit change in gre, the log odds of admission (versus
non-admission) increases by 0.002.
* For a one unit increase in gpa, the log odds of being admitted to graduate school
increases by 0.804.
* rank The indicator variables for rank have a slightly dierent interpretation. For example, having attended an undergraduate institution with rank of 2, versus an institution
with a rank of 1, changes the log odds of admission by -0.675.
* Below the table of coefficients are fit indices, including the null and deviance residuals
and the AIC.


### Confidence Intervals for Estimates
* We can use the confint function to obtain confidence intervals for the coefficient esti-
mates.
* Note that for logistic models, confidence intervals are based on the profiled log-
likelihood function.
* We can also get confidence Intervals based on just the standard errors by using the
default method.

In [ ]:

## CIs using profiled log-likelihood
confint(model1)
## Waiting for profiling to be done...
## 2.5 % 97.5 %
## (Intercept) -6.271620 -1.79255
## gre 0.000138 0.00444
## gpa 0.160296 1.46414
## rank2 -1.300889 -0.05675
## rank3 -2.027671 -0.67037
## rank4 -2.400027 -0.75354
## CIs using standard errors

In [ ]:

confint.default(model1)


### The Wald Test (aod package)- optional
* We can test for an overall effect of rank using the wald.test function of the aod
pacakge.
* ***aod: Analysis of Overdispersed Data***. This package provides a set of functions to analyse
overdispersed counts or proportions
* The order in which the coefficients are given in the table of coefficients is the same as
the order of the terms in the model. This is important because the **``wald.test()``** function
refers to the coefficients by their order in the model. We use the **``wald.test()``** function.
* b supplies the coefficients, while Sigma supplies the variance covariance matrix of the
error terms, finally Terms tells R which terms in the model are to be tested, in this
case, terms 4, 5, and 6, are the three terms for the levels of rank.


In [ ]:
wald.test(b = coef(model1), Sigma = vcov(model1), Terms = 4:6)
## Wald test:
## ----------
##
## Chi-squared test:
## X2 = 20.9, df = 3, P(> X2) = 0.00011

In [ ]:

 The chi-squared test statistic of 20.9, with three degrees of freedom is associated with a
p-value of 0.00011 indicating that the overall eect of rank is statistically signicant.
32

In [ ]:
### Odds and Odds Ratio
 You can also exponentiate the coecients and interpret them as odds-ratios.
## odds ratios only
exp(coef(model1))
## (Intercept) gre gpa rank2 rank3 rank4
## 0.0185 1.0023 2.2345 0.5089 0.2618 0.2119
 We can use the same idea to get odds ratios and their condence intervals, by exponen-
tiating the condence intervals from before. To put it all in one table, we use cbind to
bind the coecients and condence intervals column-wise.

In [ ]:

## odds ratios and 95% CI
exp(cbind(OR = coef(model1), confint(myodel1)))
## Waiting for profiling to be done...
## OR 2.5 % 97.5 %
## (Intercept) 0.0185 0.00189 0.167
## gre 1.0023 1.00014 1.004
## gpa 2.2345 1.17386 4.324
## rank2 0.5089 0.27229 0.945
## rank3 0.2618 0.13164 0.512
## rank4 0.2119 0.09072 0.471




In [ ]:
 Now we can say that for a one unit increase in gpa, the odds of being admitted to
graduate school (versus not being admitted) increase by a factor of 2.23. Note that
while R produces it, the odds ratio for the intercept is not generally interpreted.
33

In [ ]:
### Using the Model to make predictions
 You can also use predicted probabilities to help you understand the model.
 Predicted probabilities can be computed for both categorical and continuous predictor
variables.
 In order to create predicted probabilities we rst need to create a new data frame with
the values we want the independent variables to take on to create our predictions.
 We will start by calculating the predicted probability of admission at each value of
rank, holding gre and gpa at their means.
 First we create and view the data frame.


In [ ]:
newdata1 <- with(mydata, data.frame(gre = mean(gre),
gpa = mean(gpa),
rank = factor(1:4)))
## view data frame
newdata1
## gre gpa rank
## 1 588 3.39 1
## 2 588 3.39 2
## 3 588 3.39 3
## 4 588 3.39 4

* Naming Conventions: These objects must have the same names as the variables in
your logistic regression model (e.g. in this example the mean for gre must be named
gre).
* Now that we have the data frame we want to use to calculate the predicted probabilities,
we can tell R to create the predicted probabilities.

* The newdata1$rankP tells R that we want to create a new variable in the dataset (data
frame) newdata1 called rankP, the rest of the command tells R that the values of rankP
should be predictions made using the predict( ) function.
* The options within the parentheses specifies that the predictions should be based on
the analysis mylogit with values of the predictor variables coming from newdata1 and
that the type of prediction is a predicted probability (type="response").

In [ ]:

 The second line of the code lists the values in the data frame newdata1.

In [ ]:

newdata1$rankP <- predict(model1, newdata = newdata1, type = "response")
newdata1
## gre gpa rank rankP
## 1 588 3.39 1 0.517
## 2 588 3.39 2 0.352
## 3 588 3.39 3 0.219
## 4 588 3.39 4 0.185


In [ ]:
### Interpretation
In this output we see that the predicted probability of being accepted into a gradu-
ate program is 0.52 for students from the highest prestige undergraduate institutions
(rank=1), and 0.18 for students from the lowest ranked institutions (rank=4), holding
gre and gpa at their means.
